Application using altimetry data obtained from AVISO.
* Region of interest: Mediterranean Sea.
* Period of interest: May 2014.
* Satellite avaible: AltiKa, Cryosat, H2 and JASON-2. 

In [ ]:
using divand
using NetCDF
using PyPlot
using Logging
Logging.configure(level=DEBUG);

INFO: Recompiling stale cache file /home/ctroupin/.julia/v0.6/lib/v0.6/NetCDF.ji for module NetCDF.


# User inputs

## Domain of interest, resolution

In [ ]:
lonmin, lonmax, latmin, latmax = -7., 40., 30., 48.
dx, dy, dt = .125, .125, 1.
lonr = lonmin:dx:lonmax;
latr = latmin:dy:latmax;
timer = 23520:dt:23521;

## File and directories

In [ ]:
datadir = joinpath("../data/");
datafile = joinpath(datadir, "ADT_20140515_20140604.dat")
resultdir = joinpath("../results/divand/");
resultfile = joinpath(resultdir, "ADT_20140515_20140604_0pt125d1.nc");
info("Data file:\n", datafile)
info("Result file:\n", resultfile)

# Read data file (4-column, ascii)

In [4]:
function readAltimetryFile(fname)

    data = readlines(open(fname,"r"))
    nobs = length(data)

    lon = zeros(nobs)
    lat = zeros(nobs)
    time = zeros(nobs)
    field = zeros(nobs)

    for i in 1:nobs
        rec = split(data[i])
        lon[i] = parse(Float64,rec[1])
        lat[i] = parse(Float64,rec[2])
        time[i] = parse(Float64,rec[3])
        field[i] = parse(Float64,rec[4])
    end
    
    return lon, lat, time, field
end
info("Data read from file ", datafile)

20-juil. 10:42:06:INFO:root:Data read from file ../data/ADT_20140515_20140604.dat


In [5]:
lon, lat, time, field = readAltimetryFile(datafile);
info("Reading ", length(lon), " data points")
info("Mean field value = ", mean(field))

20-juil. 10:42:06:INFO:root:Reading 11371 data points
20-juil. 10:42:06:INFO:root:Mean field value = -0.09550065957259697


Create mask and metrics using coordinate vectors

In [6]:
mask, (px, py, pt), (xi, yi, ti) = divand_rectdom(lonr, latr, timer);

In [7]:
size(mask)

(377, 145, 2)

## Analysis parameters

In [8]:
# correlation length
len = (1., 1., 4.);
# obs. error variance normalized by the background error variance
epsilon2 = 1.;

# Perform interpolation

In [9]:
@time fi,s = divandrun(mask, (px, py, pt), (xi, yi, ti), (lon, lat, time), field, len, epsilon2; alphabc=2);

 21.470195 seconds (19.65 M allocations: 4.255 GiB, 10.04% gc time)


## Export results to netCDF

In [11]:
info("Writing interpolated field in file:\n", resultfile)
sz = size(mask)
dims = [NcDim("longitude",sz[1]), NcDim("latitude",sz[2]), NcDim("time",sz[3])];
nc = NetCDF.create(resultfile, NcVar("ADT", dims))
nc["ADT"][:,:, :] = fi
NetCDF.close(nc);

20-juil. 10:44:20:INFO:root:Writing interpolated field in file:
../results/divand/ADT_20140515_20140604_0pt125d1.nc


LoadError: [91mNetCDF error code 13:
	Permission denied[39m

# Plotting

Done in [another notebook in Python](../python/plot_AVISO_divand.ipynb) due to a problem with Julia / matplotlib.